## This notebook handles mapping srprec to census tract, and then adding on census data

In [ ]:
# imports
import pandas as pd
import requests
import pickle as pkl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [7]:
data18 = pd.read_pickle('./data/trend18.pkl')
data18.shape

(1334, 26)

In [8]:
data18.head()

,county,srprec,cddist,TOTREG,TOTVOTE,CNGDEM01,CNGREP01,election,type,totreg_r,...,nlp,grn,ref,dcl,male,female,hispdem,hisprep,hispdcl,hispoth
0,30,10316,47,1735,1278,630,592,g18,V,1238.0,...,0.0,2.0,0.0,265.0,582.0,656.0,55.0,43.0,30.0,6.0
1,30,10317,47,2079,1214,638,513,g18,V,1184.0,...,0.0,4.0,0.0,286.0,588.0,596.0,165.0,60.0,71.0,10.0
2,30,10319,47,1448,999,495,462,g18,V,945.0,...,0.0,2.0,2.0,256.0,443.0,502.0,68.0,37.0,49.0,5.0
3,30,10320,47,2407,1686,847,758,g18,V,1644.0,...,0.0,5.0,1.0,423.0,851.0,793.0,79.0,57.0,47.0,7.0
4,30,10325,47,1465,1035,488,524,g18,V,1012.0,...,0.0,2.0,1.0,189.0,488.0,524.0,67.0,38.0,18.0,1.0


In [76]:
data18['srprec'].value_counts()

2048     1
2095     1
45352    1
2088     1
2049     1
        ..
7182     1
2033     1
2034     1
2036     1
2047     1
Name: srprec, Length: 1334, dtype: Int64

### Select census data

Voting data contains 'srprec' or voting precints.  
Statewidedatabase.org provides conversion tables between precints and census tracts.  

Bring in census DP03 data and select columns.

In [51]:
dp03_18 = pd.read_pickle('./census_data/DP03_clean.pkl')
dp03_18.shape

(583, 143)

In [69]:
dp03_18.head(2)

,Geographic Area Name,GEO_ID,Estimate EMPLOYMENT STATUS Population 16 years and over,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces,Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force,Estimate EMPLOYMENT STATUS Civilian labor force,...,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL All people Under 18 years Related children of the householder under 18 years Related children of the householder 5 to 17 years,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL All people 18 years and over,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL All people 18 years and over 18 to 64 years,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL All people 18 years and over 65 years and over,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL People in families,Estimate PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL Unrelated individuals 15 years and over,Geographic Area Name,state,county,tract
1,"Census Tract 626.44, Orange County, California",1400000US06059062644,6767,3967,3967,3811,156,0,2800,3967,...,-888888888.0,-888888888.0,-888888888.0,-888888888.0,-888888888.0,-888888888.0,"Census Tract 626.44, Orange County, California",06,059,062644
2,"Census Tract 626.40, Orange County, California",1400000US06059062640,3062,2446,2436,2320,116,10,616,2436,...,-888888888.0,-888888888.0,-888888888.0,-888888888.0,-888888888.0,-888888888.0,"Census Tract 626.40, Orange County, California",06,059,062640


In [53]:
dp03_18['Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars)'].describe()

count            583
unique           574
top       -666666666
freq               3
Name: Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars), dtype: object

In [38]:
dp03_18.columns.tolist()

['Geographic Area Name',
 'GEO_ID',
 'Estimate EMPLOYMENT STATUS Population 16 years and over',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces',
 'Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force',
 'Estimate EMPLOYMENT STATUS Civilian labor force',
 'Estimate EMPLOYMENT STATUS Civilian labor force Unemployment Rate',
 'Estimate EMPLOYMENT STATUS Females 16 years and over',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force Civilian labor force',
 'Estimate

In [54]:
dp03_cols = ['Geographic Area Name',
             'GEO_ID',
             'tract',
             'Estimate EMPLOYMENT STATUS Population 16 years and over',
             'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force',
             'Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force', 
             'Estimate OCCUPATION Civilian employed population 16 years and over Management, business, science, and arts occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Service occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Sales and office occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Natural resources, construction, and maintenance occupations',
             'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars)'
            ]

In [55]:
dp03_18sub = dp03_18[dp03_cols]

In [68]:
dp03_18sub.head(2)

,Geographic Area Name,Geographic Area Name,GEO_ID,tract,Estimate EMPLOYMENT STATUS Population 16 years and over,Estimate EMPLOYMENT STATUS Population 16 years and over In labor force,Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force,"Estimate OCCUPATION Civilian employed population 16 years and over Management, business, science, and arts occupations",Estimate OCCUPATION Civilian employed population 16 years and over Service occupations,Estimate OCCUPATION Civilian employed population 16 years and over Sales and office occupations,"Estimate OCCUPATION Civilian employed population 16 years and over Natural resources, construction, and maintenance occupations",Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars)
1,"Census Tract 626.44, Orange County, California","Census Tract 626.44, Orange County, California",1400000US06059062644,062644,6767,3967,2800,2547,262,819,91,146953
2,"Census Tract 626.40, Orange County, California","Census Tract 626.40, Orange County, California",1400000US06059062640,062640,3062,2446,616,1202,287,563,63,84632


In [60]:
dp03_18sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 583
Data columns (total 12 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                           --------------  ----- 
 0   Geographic Area Name                                                                                                             583 non-null    object
 1   Geographic Area Name                                                                                                             583 non-null    object
 2   GEO_ID                                                                                                                           583 non-null    object
 3   tract                                                                                                    

In [58]:
num_cols = ['Estimate EMPLOYMENT STATUS Population 16 years and over',
             'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force',
             'Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force', 
             'Estimate OCCUPATION Civilian employed population 16 years and over Management, business, science, and arts occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Service occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Sales and office occupations',
             'Estimate OCCUPATION Civilian employed population 16 years and over Natural resources, construction, and maintenance occupations',
             'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars)']

In [65]:
for col in num_cols:
    dp03_18sub[col] = pd.to_numeric(dp03_18sub[col])

# will generate a warning (not an error) about making a change a copy of a slide of a dataframe.  
# we will just proceed.  

<ipython-input-65-41ea6c34ecb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dp03_18sub[col] = pd.to_numeric(dp03_18sub[col])


In [67]:
dp03_18sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 583
Data columns (total 12 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                           --------------  ----- 
 0   Geographic Area Name                                                                                                             583 non-null    object
 1   Geographic Area Name                                                                                                             583 non-null    object
 2   GEO_ID                                                                                                                           583 non-null    object
 3   tract                                                                                                    

In [77]:
dp03_18sub['tract'].value_counts()

086404    1
099229    1
063702    1
110202    1
063202    1
         ..
099602    1
021913    1
032045    1
001403    1
042103    1
Name: tract, Length: 583, dtype: int64

### Map srprec to census tract

In [71]:
convert18 = pd.read_csv('./map_files/c059_g18_sr_blk_map.csv')
convert18.shape

(22985, 8)

In [72]:
convert18.head()

,srprec,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
0,NaN,0,0,20,NaN,NaN,20,100.000000
1,10316.0,110106,2000,11,1720.0,0.639535,11,100.000000
2,10316.0,110106,2001,15,1720.0,0.872093,15,100.000000
3,10316.0,110106,2002,62,1720.0,3.604651,62,100.000000
4,10316.0,110106,2003,10,1720.0,0.581395,21,47.619048


In [75]:
convert18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22985 entries, 0 to 22984
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   srprec     22984 non-null  float64
 1   tract      22985 non-null  int64  
 2   block      22985 non-null  int64  
 3   blkreg     22985 non-null  int64  
 4   srtotreg   22984 non-null  float64
 5   pctsrprec  22984 non-null  float64
 6   blktotreg  22985 non-null  int64  
 7   pctblk     22985 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 1.4 MB


In [79]:
convert18['tract'].nunique()

583

In [86]:
convert18['srprec'].nunique()

1334

In [81]:
convert18.groupby('tract').count()

,srprec,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
tract,,,,,,,
0,0,1,1,0,0,1,1
1101,66,66,66,66,66,66,66
1102,35,35,35,35,35,35,35
1103,33,33,33,33,33,33,33
1201,37,37,37,37,37,37,37
...,...,...,...,...,...,...,...
110604,87,87,87,87,87,87,87
110605,43,43,43,43,43,43,43
110606,20,20,20,20,20,20,20


In [85]:
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
convert18.groupby('srprec').count()

,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
srprec,,,,,,,
2001.0,31,31,31,31,31,31,31
2002.0,7,7,7,7,7,7,7
2008.0,32,32,32,32,32,32,32
2009.0,28,28,28,28,28,28,28
2011.0,14,14,14,14,14,14,14
...,...,...,...,...,...,...,...
75116.0,18,18,18,18,18,18,18
75117.0,2,2,2,2,2,2,2
75122.0,3,3,3,3,3,3,3


In [88]:
convert18.loc[convert18['srprec'] == 75117.0]

,srprec,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
22942,75117.0,75810,1006,64,99.0,64.646465,64,100.0
22943,75117.0,75810,1016,35,99.0,35.353535,35,100.0


In [89]:
convert18.loc[convert18['tract'] == 110106]

,srprec,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
1,10316.0,110106,2000,11,1720.0,0.639535,11,100.000000
2,10316.0,110106,2001,15,1720.0,0.872093,15,100.000000
3,10316.0,110106,2002,62,1720.0,3.604651,62,100.000000
4,10316.0,110106,2003,10,1720.0,0.581395,21,47.619048
5,10316.0,110106,2009,6,1720.0,0.348837,45,13.333333
...,...,...,...,...,...,...,...,...
333,10701.0,110106,3014,56,1582.0,3.539823,56,100.000000
334,10701.0,110106,3015,30,1582.0,1.896334,30,100.000000
335,10701.0,110106,3016,60,1582.0,3.792668,60,100.000000
336,10701.0,110106,3017,76,1582.0,4.804046,76,100.000000
